# Imports

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

In [3]:
import lightgbm as lgbm
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, train_test_split

In [4]:
PATH = 'data/nyc-taxi/'
CSV_PATH = 'submissions/LGBM/'

In [5]:
train_df = pd.read_feather('tmp/taxi-train-clean2')
test_df = pd.read_feather('tmp/taxi-test-clean2')
test_df_raw = pd.read_csv(f'{PATH}test.csv', usecols=['key'])

/home/szelee/anaconda3/envs/tf/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [6]:
train_df.shape, test_df.shape

((19503734, 43), (9914, 42))

In [7]:
%%time
X_train, X_valid, y_train, y_valid = train_test_split(train_df.drop('fare_amount', axis=1), train_df['fare_amount'], test_size=0.1, random_state=111)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

CPU times: user 25.1 s, sys: 5.4 s, total: 30.5 s
Wall time: 25.8 s


In [8]:
params = {
        'boosting_type':'gbdt',
        'objective': 'regression',
        'nthread': 16,
        'num_leaves': 31,
        'learning_rate': 0.05,
        'max_depth': -1,
        'subsample': 0.8,
        'bagging_fraction' : 1,
        'max_bin' : 5000 ,
        'bagging_freq': 20,
        'colsample_bytree': 0.6,
        'metric': 'rmse',
        'min_split_gain': 0.5,
        'min_child_weight': 1,
        'min_child_samples': 10,
        'scale_pos_weight':1,
        'zero_as_missing': True,
        'seed':0
    }

In [9]:
matrix_train = lgbm.Dataset(X_train, y_train)
matrix_test = lgbm.Dataset(X_valid, y_valid)

model=lgbm.train(params=params,
                train_set=matrix_train,
                num_boost_round=10_000, 
                early_stopping_rounds=500,
                verbose_eval=500,
                valid_sets=matrix_test)

Training until validation scores don't improve for 500 rounds.
[500]	valid_0's rmse: 3.47572
[1000]	valid_0's rmse: 3.43611
[1500]	valid_0's rmse: 3.42125
[2000]	valid_0's rmse: 3.4116
[2500]	valid_0's rmse: 3.40573
[3000]	valid_0's rmse: 3.40085
[3500]	valid_0's rmse: 3.39726
[4000]	valid_0's rmse: 3.39447
[4500]	valid_0's rmse: 3.39148
[5000]	valid_0's rmse: 3.38949
[5500]	valid_0's rmse: 3.38816
[6000]	valid_0's rmse: 3.38702
[6500]	valid_0's rmse: 3.38631
[7000]	valid_0's rmse: 3.38543
[7500]	valid_0's rmse: 3.38441
[8000]	valid_0's rmse: 3.38388
[8500]	valid_0's rmse: 3.38356
[9000]	valid_0's rmse: 3.38298
[9500]	valid_0's rmse: 3.3819
[10000]	valid_0's rmse: 3.38178
Did not meet early stopping. Best iteration is:
[9985]	valid_0's rmse: 3.38178


In [10]:
prediction = model.predict(test_df, num_iteration = model.best_iteration) 
 
submission = pd.DataFrame(
    {'key': test_df_raw.key, 'fare_amount': prediction},
    columns = ['key', 'fare_amount'])

submission.to_csv(f'{CSV_PATH}lgbm_2b.csv', index = False)

In [11]:
matrix_train = lgbm.Dataset(X_train, y_train)
matrix_test = lgbm.Dataset(X_valid, y_valid)

model2=lgbm.train(params=params,
                train_set=matrix_train,
                num_boost_round=10_000, 
                early_stopping_rounds=500,
                verbose_eval=500,
                valid_sets=matrix_test)

Training until validation scores don't improve for 500 rounds.


KeyboardInterrupt: 

In [ ]:
predictio2n = model2.predict(test_df, num_iteration = model2.best_iteration) 
 
submission2 = pd.DataFrame(
    {'key': test_df_raw.key, 'fare_amount': prediction},
    columns = ['key', 'fare_amount'])

submission2.to_csv(f'{CSV_PATH}lgbm_3b.csv', index = False)